# Constructing Datasets from a Factor Model

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from satis_models import sharpe_ratio
from factor_models import linear_factor_mod
from finrl.config import config
from pandas.tseries.offsets import BDay
import scipy
%load_ext autoreload

In [2]:
DB_PATH = os.path.join('C:\\', 'Users','matus','gdrive','datasets', 'Research')
ESTIMATION_WINDOW = 36 #3 years estimation period

DB_NAME = '00_db_SPX'
DB_FACTORS = '00_db_FACTORS__PX_LAST.xlsx'

DB_FNAMES = ['GROWTH50', 'MOM50', 'SIZE50']

In [3]:
db_full = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '__PX_LAST.csv'), index_col = 0, parse_dates = True)
db_m = db_full.resample('M').ffill()
db_lrets = np.log1p(db_m.pct_change())
db_lrets = db_lrets.iloc[-ESTIMATION_WINDOW:]
db_lrets = db_lrets.loc[:,~db_lrets.iloc[0].isna()]

In [4]:
db_factors = pd.read_excel(os.path.join(DB_PATH, DB_FACTORS), index_col = 0, parse_dates = True)
db_factors_m = db_factors.dropna().resample('M').ffill()
db_factors_m = np.log1p(db_factors_m.pct_change())
db_factors_m = db_factors_m.iloc[-ESTIMATION_WINDOW:]

# Making Factor Datasets

In [5]:
alpha, beta, eps = linear_factor_mod(db_lrets*100, db_factors_m*100, regularize = 'net', return_alpha=True)

In [6]:
loadings = pd.DataFrame(beta,columns = db_factors.columns, index = db_lrets.columns)

In [7]:
loadings

,PVALUEUS Index,PMOMENUS Index,PSIZEUS Index,SPX Index
A UN Equity,1.180663,-0.210399,1.341714,0.876855
AAL UW Equity,2.324936,-0.662918,-0.238527,1.726496
AAP UN Equity,1.267061,-0.220220,-4.676084,0.924405
AAPL UW Equity,-0.435322,0.623500,1.725781,1.453615
ABBV UN Equity,-1.108157,-0.993623,-0.891483,0.652763
...,...,...,...,...
YUM UN Equity,-0.939574,0.246497,-1.422366,1.136680
ZBH UN Equity,-0.417244,1.170789,-0.923192,1.535019
ZBRA UW Equity,-0.129286,-0.034303,-3.916881,1.397786
ZION UW Equity,2.887831,-0.874634,-5.755810,1.044340


In [98]:
growth_50 = loadings.sort_values('PVALUEUS Index').index[:50]
mom_50 = loadings.sort_values('PMOMENUS Index').index[-50:]
size_50 = loadings.sort_values('PSIZEUS Index').index[:50]

In [99]:
o = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '__PX_OPEN.csv'), index_col = 0)
h = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '__PX_HIGH.csv'), index_col = 0)
l = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '__PX_LOW.csv'), index_col = 0)
c = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '__PX_LAST.csv'), index_col = 0)
v = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '__PX_VOLUME.csv'), index_col = 0)

In [86]:
def make_dataset(o ,h ,l ,c ,v ,fltr, name):
    o = o.reindex(fltr, axis = 1).to_csv(os.path.join(DB_PATH, DB_NAME + '_' + name + '__PX_OPEN.csv'))
    h = h.reindex(fltr, axis = 1).to_csv(os.path.join(DB_PATH, DB_NAME + '_' + name + '__PX_HIGH.csv'))
    l = l.reindex(fltr, axis = 1).to_csv(os.path.join(DB_PATH, DB_NAME + '_' + name + '__PX_LOW.csv'))
    c = c.reindex(fltr, axis = 1).to_csv(os.path.join(DB_PATH, DB_NAME + '_' + name + '__PX_LAST.csv'))
    v = v.reindex(fltr, axis = 1).to_csv(os.path.join(DB_PATH, DB_NAME + '_' + name + '__PX_VOLUME.csv'))
    print('Dataset ' + name + ' saved in ' + DB_PATH)

In [87]:
make_dataset(o, h, l, c, v, growth_50, 'GROWTH50')

Dataset GROWTH50 saved in C:\Users\matus\gdrive\datasets\Research


In [88]:
make_dataset(o, h, l, c, v, mom_50, 'MOM50')

Dataset MOM50 saved in C:\Users\matus\gdrive\datasets\Research


In [89]:
make_dataset(o, h, l, c, v, size_50, 'SIZE50')

Dataset SIZE50 saved in C:\Users\matus\gdrive\datasets\Research


# Combined Factors Datasets

In [100]:
def make_two_factor_dataset(db_name_1, db_name_2, db_merged_name : str) :
    #read first factor dataset
    o1 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_1 + '__PX_OPEN.csv'), index_col = 0)
    h1 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_1 + '__PX_HIGH.csv'), index_col = 0)
    l1 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_1 + '__PX_LOW.csv'), index_col = 0)
    c1 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_1 + '__PX_LAST.csv'), index_col = 0)
    v1 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_1 + '__PX_VOLUME.csv'), index_col = 0)
    
#     print(o1.columns)
    #read second factor dataset
    o2 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_2 + '__PX_OPEN.csv'), index_col = 0)
    h2 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_2 + '__PX_HIGH.csv'), index_col = 0)
    l2 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_2 + '__PX_LOW.csv'), index_col = 0)
    c2 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_2 + '__PX_LAST.csv'), index_col = 0)
    v2 = pd.read_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_name_2 + '__PX_VOLUME.csv'), index_col = 0)
#     print(o2.columns)
    o_m = pd.concat([o1,o2], axis = 1)
    o_m = o_m.T.drop_duplicates().T
    
    h_m = pd.concat([h1,h2], axis = 1)
    h_m = h_m.T.drop_duplicates().T
    
    l_m = pd.concat([l1,l2], axis = 1)
    l_m = l_m.T.drop_duplicates().T
    
    c_m = pd.concat([c1,c2], axis = 1)
    c_m = c_m.T.drop_duplicates().T
    
    v_m = pd.concat([v1,v2], axis = 1)
    v_m = v_m.T.drop_duplicates().T
    
    print(f'There are {100 - len(o_m.columns.unique())} duplicate factor stocks. in {db_merged_name}')
    
    #save
    o_m.to_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_merged_name + '__PX_OPEN.csv'))
    h_m.to_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_merged_name + '__PX_HIGH.csv'))
    l_m.to_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_merged_name + '__PX_LOW.csv'))
    c_m.to_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_merged_name + '__PX_LAST.csv'))
    v_m.to_csv(os.path.join(DB_PATH, DB_NAME + '_' + db_merged_name + '__PX_VOLUME.csv'))
    
    print('Dataset ' + db_merged_name + ' saved in ' + DB_PATH)
    
    

In [101]:
make_two_factor_dataset(DB_FNAMES[0], DB_FNAMES[2], 'GRSIZE100')

There are 6 duplicate factor stocks. in GRSIZE100
Dataset GRSIZE100 saved in C:\Users\matus\gdrive\datasets\Research


In [102]:
make_two_factor_dataset(DB_FNAMES[1], DB_FNAMES[2], 'MOMSIZE100')

There are 3 duplicate factor stocks. in MOMSIZE100
Dataset MOMSIZE100 saved in C:\Users\matus\gdrive\datasets\Research
